# Coalescent simulations with msprime

In [1]:
import msprime as ms
import numpy as np
import toytree
import toyplot

### Coalescent simulations 
A coalescent tree represents the genealogical history of a sample of  orthologous gene copies in a population. All orthologs are related by a pattern of descent from ancestors to descendants. In a Wright-Fisher process model any tree **topology** (branching pattern) is equally likely to describe the relationships among a set of samples in a population. However, the **times** of coalescent events are not random, and are affected by demographic parameters. 

As we will see later these coalescent times will affect the expected distribution of genetic diversity among sampled copies at a locus. However, before we get to mutations and genetic diversity let's look first just at coalescent genealogies.

**Note: in msprime Ne represents the diploid effective population size, so the effective number of haploid gene copies in the population is 2N. Sample size is the number of haploid gene copies sampled.**

### generate a coalescent tree for a single locus
Here we are calling `.simulate()` from msprime to sample 10 loci from a population with effective popultaion size of 1000. Execute this code cell multiple times. You will see that the genealogy changes each time, owing to the randomly sampled coalescent times that are sampled to represent common ancestors in each simulation. Try changing the value of `Ne` in the `.simulate()` function to a higher or lower integer. What happens to the scalebar on the left representing coalescent times in number of generations?


In [31]:
# simulate a tree_sequence (includes only 1 tree for now)
tree_sequence = ms.simulate(sample_size=10, Ne=1000)

# draw the tree (execute several times to see different trees)
tree = next(tree_sequence.trees())
toytree.tree(tree.newick()).draw(layout='down', scalebar=True, width=300);

<svg class="toyplot-canvas-Canvas" height="260.0px" id="ta4728161171c4f51a0eb1ac0efc2b278" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 300.0 260.0" width="300.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 5 3 7 8 6 10 9 2 4 1 0 1468 2937

### generate multiple independent coalescent trees
The relationships among the sampled tips is random. The average time to the most recent common ancestor is not random, but determined by Ne. Below I plot 5 replicate simulations on the same y-axis to demonstrate the variation in coalescent times for a sample of 8 individuals from a population with Ne=5000. 

In [33]:
# a generator returns num_replicates independent simulations
tree_generator = ms.simulate(sample_size=8, Ne=5000, num_replicates=5)

# get a tree_sequence (includes only 1 tree for now) for each replicate simulation
tree_seqs = [next(ts.trees()) for ts in tree_generator]

# get newick string of the tree from each tree_sequence
trees = [i.newick() for i in tree_seqs]

# draw the trees from all simulations
mtre = toytree.mtree(trees)
mtre.draw_tree_grid(shared_axis=True);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t369996386d164141ba270091c37d2313" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 800.0 250.0" width="800.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 7 6 3 8 2 1 5 4 8 1 6 7 5 3 4 2 6 5 8 2 1 7 4 3 7 1 5 4 8 2 6 3 6 4 3 5 2 8 1 7 0 20443 40886

### measure coalescent times

The mean waiting time until a coalescence event occurs among a sample of individuals from a population is:

$$ Pr(coalescence~given~n~lineages) = \frac{n(n-1)}{2}\frac{1}{2N} $$ 

The first term is the number of pairs of lineages and the second term is the probability of a given pair coalescing. Each pair is considered to coalesce independently. This is a Markov process, once one coalescent event occurs it is like starting again with n-1 sequences. 

Below we simulate the coalescent time for two randomly samples from a population 10K times which converges towards the answer of 2N for a population size of 1000. 

In [94]:
# set up simulations for 100K replicates with pop size of 1000
tree_generator = ms.simulate(sample_size=8, Ne=1000, num_replicates=10000)
tree_seqs = [next(ts.trees()) for ts in tree_generator]

# draw two random tips (1-6)
tips = np.random.choice(range(1, 7), 2, replace=False)

# get coalescent times for these two samples across all replicate sims
tmrcas = [t.get_tmrca(*tips) for t in tree_seqs]

# print mean, std
mean = np.mean(tmrcas)
std = np.std(tmrcas)
print("coalescent time: mean={:.0f}; std={:.0f}".format(mean, std))

coalescent time: mean=2002; std=1993


### generate multiple neighboring coalescent trees
These trees are not independent of each other because they share many of the same ancestors. This occurs between loci that are linked, for example those that are located next to each other in the genome. Here I simulate a 10000 bp chunk of the genome with recombination. When a recombination crossover occurs it 'splits' the history of the locus so that there are more genealogies required to trace the history of all parts of the locus. 

In [95]:
# simulate a tree_sequence (includes only 1 tree for now)
tree_sequence = ms.simulate(sample_size=6, Ne=100000, recombination_rate=1e-4)

# get newick string from each tree_seq
trees = [i.newick() for i in tree_sequence.trees()]

# draw the tree (execute several times to see different trees)
mtre = toytree.mtree(trees)
mtre.draw_tree_grid(nrows=1, ncols=5, shared_axis=True);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t82fe1eb3e84c4a41ababbca1cc20d64a" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 800.0 250.0" width="800.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 6 1 2 3 5 4 6 1 2 5 3 4 6 1 2 3 5 4 6 1 2 3 5 4 6 1 2 3 5 4 0 98008 196016

### Structured coalescent
We can describe more complex scenarios in which population structure exists, such that all samples do not have the same probability of coalescing. The structured coalescent involves **defining populations** and parameters affecting those populations like migration or a divergence time for when they split from a common ancestor. This is a more complex type of demographic model. 

In the case where two populations diverged from a common ancestor without subsequent migration, we can simply define this as three distinct single population coalescent probabilities. Below we simulate two populations that diverged 20K generations ago. The coalescent times among samples within either population are very short since each population size is quite small (1000), but the final coalescence of the two lineages cannot occur until at least 20K generations ago, leading to long branches for this event. 

You can see that the *topology is no longer random* in the structured coalescent. Now samples 1-3 from the first population group together more often with each other than with samples 4-9.

In [97]:
# define two populations and number of samples from each
populations = [
    ms.PopulationConfiguration(sample_size=3, initial_size=1000),
    ms.PopulationConfiguration(sample_size=6, initial_size=1000),
]

# demographic events (populations divergence time in generations)
demography = [
    ms.MassMigration(time=20000, source=0, dest=1)
]

# migration between populations (set to 0)
migmat = np.zeros((2,2), dtype=int).tolist()

# get simulator
tree_generator = ms.simulate(
    migration_matrix=migmat,
    population_configurations=populations,
    demographic_events=demography,
    num_replicates=5,
)

# get a tree_sequence (includes only 1 tree for now) for each replicate simulation
tree_seqs = [next(ts.trees()) for ts in tree_generator]

# get newick string of the tree from each tree_sequence
trees = [i.newick() for i in tree_seqs]

# draw the trees from all simulations
mtre = toytree.mtree(trees)
mtre.draw_tree_grid(shared_axis=True);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t64138997b4744fb6aea8ed33a6bba0e2" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 800.0 250.0" width="800.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 6 5 8 9 4 7 3 1 2 9 6 8 4 7 5 3 2 1 9 5 8 4 7 6 3 2 1 9 5 6 8 4 7 3 2 1 6 5 9 7 8 4 2 1 3 0 11544 23087

### Incomplete lineage sorting

The process of incomplete lineage sorting describes when the coalescent times for gene copies span farther back in time than a population or species divergence event such that the genealogies do not match the divergence pattern. Given large population sizes *incomplete lineage sorting is expected to occur*. The frequency with which it is observed across the genome is informative about the size of recent and ancestral populations. 

Here I simulate data similar to above with the only difference being the population sizes (Ne) of the two subpopulations, increased from 1K to 20K. We now see a lot of incomplete lineage sorting where samples from either defined population (1-3) or (4-9) do not always group together into a monophyletic clade. The population structure still has some effect on coalescent events that occur before 20K generations ago (the pop divergence time) but it does not affect coalescent events that happen further back in time since there is no longer structure between the sampled copies at that time. 


In [98]:
# define two populations and number of samples from each
populations = [
    ms.PopulationConfiguration(sample_size=3, initial_size=20000),
    ms.PopulationConfiguration(sample_size=6, initial_size=20000),
]

# demographic events (populations divergence time in generations)
demography = [
    ms.MassMigration(time=20000, source=0, dest=1),
]

# migration between populations (set to 0)
migmat = np.zeros((2,2), dtype=int).tolist()

# get simulator
tree_generator = ms.simulate(
    migration_matrix=migmat,
    population_configurations=populations,
    demographic_events=demography,
    num_replicates=5,
)

# get a tree_sequence (includes only 1 tree for now) for each replicate simulation
tree_seqs = [next(ts.trees()) for ts in tree_generator]

# get newick string of the tree from each tree_sequence
trees = [i.newick() for i in tree_seqs]

# draw the trees from all simulations
mtre = toytree.mtree(trees)
mtre.draw_tree_grid(shared_axis=True);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t4b555d5a721c4c018add47febb0643e9" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 800.0 250.0" width="800.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 7 4 8 9 5 6 3 2 1 9 6 7 4 3 2 1 8 5 8 7 9 6 5 2 1 4 3 2 1 6 9 4 8 7 5 3 8 6 5 2 7 4 9 3 1 0 49727 99455

### Mutations
msprime allow us to drop mutations onto simulated genealogies so that we can then measure the expected genetic variation within and among populations under a given demographic scenario. This mutational model is called "infinite sites" because we do not allow for back-mutations (homoplasy) which we often try to model in phylogenetics. It is a simplification but one that likely has little effect for very closely related samples. 

Here I simulate a 10Kb genomic region with no recombination (it will have only a single genealogy) and a mutation rate of 1e-8 (mutations per site per generation). 

In [106]:
# simulate a tree_sequence (includes only 1 tree for now)
tree_sequence = ms.simulate(
    sample_size=10, 
    Ne=10000,
    mutation_rate=1e-8,
    length=10000,
    random_seed=12345,
)

In [107]:
print(tree.draw_text())

  18               
┏━━┻━━┓            
┃    17            
┃  ┏━━┻━━━┓        
┃  ┃     16        
┃  ┃   ┏━━┻━━━┓    
┃ 15   ┃      ┃    
┃ ┏┻┓  ┃      ┃    
┃ ┃ ┃  ┃     14    
┃ ┃ ┃  ┃    ┏━┻━━┓ 
┃ ┃ ┃  ┃    ┃   13 
┃ ┃ ┃  ┃    ┃   ┏┻┓
┃ ┃ ┃ 12    ┃   ┃ ┃
┃ ┃ ┃ ┏┻┓   ┃   ┃ ┃
┃ ┃ ┃ ┃ ┃  11   ┃ ┃
┃ ┃ ┃ ┃ ┃ ┏━┻┓  ┃ ┃
┃ ┃ ┃ ┃ ┃ ┃ 10  ┃ ┃
┃ ┃ ┃ ┃ ┃ ┃ ┏┻┓ ┃ ┃
6 3 8 2 5 9 1 7 0 4



In [108]:
tree = tree_sequence.first()
for site in tree.sites():
    for mutation in site.mutations:
        print("Mutation @ position {:.2f} over node {}".format(
              site.position, mutation.node))

Mutation @ position 504.88 over node 4
Mutation @ position 1027.44 over node 15
Mutation @ position 1709.14 over node 8
Mutation @ position 2189.50 over node 6
Mutation @ position 2587.19 over node 6
Mutation @ position 3793.45 over node 16
Mutation @ position 4396.45 over node 3
Mutation @ position 5563.97 over node 0
Mutation @ position 6561.20 over node 15
Mutation @ position 6768.74 over node 8
Mutation @ position 7908.23 over node 8
Mutation @ position 8119.90 over node 15
Mutation @ position 8218.79 over node 14
Mutation @ position 8318.99 over node 14
Mutation @ position 8994.66 over node 14
Mutation @ position 9308.16 over node 11


In [105]:
for variant in tree_sequence.variants():
    print(
        variant.site.id, 
        variant.site.position,
        variant.alleles,
        variant.genotypes, 
        sep="\t")

0	504.87956730648875	('0', '1')	[0 0 0 0 1 0 0 0 0 0]
1	1027.4398606270552	('0', '1')	[0 0 0 1 0 0 0 0 1 0]
2	1709.142595063895	('0', '1')	[0 0 0 0 0 0 0 0 1 0]
3	2189.5004669204354	('0', '1')	[0 0 0 0 0 0 1 0 0 0]
4	2587.190617341548	('0', '1')	[0 0 0 0 0 0 1 0 0 0]
5	3793.4521259739995	('0', '1')	[1 1 1 0 1 1 0 1 0 1]
6	4396.446084138006	('0', '1')	[0 0 0 1 0 0 0 0 0 0]
7	5563.973018433899	('0', '1')	[1 0 0 0 0 0 0 0 0 0]
8	6561.195862013847	('0', '1')	[0 0 0 1 0 0 0 0 1 0]
9	6768.737088423222	('0', '1')	[0 0 0 0 0 0 0 0 1 0]
10	7908.225138671696	('0', '1')	[0 0 0 0 0 0 0 0 1 0]
11	8119.895635172725	('0', '1')	[0 0 0 1 0 0 0 0 1 0]
12	8218.789293896407	('0', '1')	[1 1 0 0 1 0 0 1 0 1]
13	8318.993039429188	('0', '1')	[1 1 0 0 1 0 0 1 0 1]
14	8994.658919982612	('0', '1')	[1 1 0 0 1 0 0 1 0 1]
15	9308.156927581877	('0', '1')	[0 1 0 0 0 0 0 1 0 1]


### Genotype matrix
This can be thought of as a sequence alignment for all variable sites over the 10Kb region that we simulated. 

In [109]:
# or we can get the matrix (sequence alignment) across
tree_sequence.genotype_matrix()

array([[0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [1, 1, 1, 0, 1, 1, 0, 1, 0, 1],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
       [1, 1, 0, 0, 1, 0, 0, 1, 0, 1],
       [1, 1, 0, 0, 1, 0, 0, 1, 0, 1],
       [1, 1, 0, 0, 1, 0, 0, 1, 0, 1],
       [0, 1, 0, 0, 0, 0, 0, 1, 0, 1]], dtype=int8)

### Allele frequency spectrum

In [110]:
# 
tree_sequence.allele_frequency_spectrum(span_normalise=False)

array([0., 8., 3., 2., 0., 3., 0., 0., 0., 0., 0.])

In [111]:
# normalized by non-variable sites
tree_sequence.allele_frequency_spectrum(span_normalise=True)

array([0.    , 0.0008, 0.0003, 0.0002, 0.    , 0.0003, 0.    , 0.    ,
       0.    , 0.    , 0.    ])

### joint site frequency spectrum

#### small populations divergence

In [112]:
# define two populations and number of samples from each
populations = [
    ms.PopulationConfiguration(sample_size=10, initial_size=1000),
    ms.PopulationConfiguration(sample_size=10, initial_size=1000),
]

# demographic events (populations divergence time in generations)
demography = [
    ms.MassMigration(time=20000, source=0, dest=1),
]

# migration between populations (set to 0)
migmat = np.zeros((2,2), dtype=int).tolist()

# get simulator
tree_sequence = ms.simulate(
    migration_matrix=migmat,
    population_configurations=populations,
    demographic_events=demography,
    mutation_rate=1e-7,
    recombination_rate=1e-7,
    length=1000000,
)

# get newick string of the tree from each tree_sequence
tree = [i.newick() for i in tree_sequence.trees()]

# draw the trees from all simulations
toytree.tree(tree[0]).draw(layout='down');

<svg class="toyplot-canvas-Canvas" height="260.0px" id="tac9230e70bae4be9824e5ec5dd504d17" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 360.0 260.0" width="360.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 17 15 18 12 11 14 13 20 19 16 5 4 10 7 8 2 9 6 3 1

In [113]:
afs = tree_sequence.allele_frequency_spectrum(
        sample_sets=[range(10), range(10,20)],
        polarised=True,
        span_normalise=False,        
)

In [114]:
import toyplot
toyplot.matrix(afs, width=500, height=500);

<svg class="toyplot-canvas-Canvas" height="500.0px" id="t64e2b6b66dfe474fa93551be6e0bc9af" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 500.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 1 2 3 4 5 6 7 8 9 10 0 0.000000 418.000000 220.000000 160.000000 116.000000 109.000000 67.000000 65.000000 45.000000 51.000000 1843.000000 1 431.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 2 205.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 3 144.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 4 126.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 5 54.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 6 70.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 7 78.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 8 53.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 9 58.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 10 1867.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000

#### large populations divergence

In [115]:
# define two populations and number of samples from each
populations = [
    ms.PopulationConfiguration(sample_size=10, initial_size=20000),
    ms.PopulationConfiguration(sample_size=10, initial_size=20000),
]

# demographic events (populations divergence time in generations)
demography = [
    ms.MassMigration(time=20000, source=0, dest=1),
]

# migration between populations (set to 0)
migmat = np.zeros((2,2), dtype=int).tolist()

# get simulator
tree_sequence = ms.simulate(
    migration_matrix=migmat,
    population_configurations=populations,
    demographic_events=demography,
    mutation_rate=1e-7,
    recombination_rate=1e-7,
    length=50000,
)

# get newick string of the tree from each tree_sequence
tree = [i.newick() for i in tree_sequence.trees()]

# draw the trees from all simulations
toytree.tree(tree[0]).draw(layout='down');

<svg class="toyplot-canvas-Canvas" height="260.0px" id="td1d0d09cc64d4073845ff518ad73e2a5" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 360.0 260.0" width="360.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 10 8 2 9 4 18 17 7 5 6 20 15 19 16 14 11 13 12 3 1

In [116]:
afs = tree_sequence.allele_frequency_spectrum(
        sample_sets=[range(10), range(10,20)],
        polarised=True,
        span_normalise=False,        
)

In [117]:
toyplot.matrix(afs, width=500, height=500);

<svg class="toyplot-canvas-Canvas" height="500.0px" id="tbdb8a435d7374c53801d0b237e42191d" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 500.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 1 2 3 4 5 6 7 8 9 10 0 0.000000 430.000000 164.000000 102.000000 57.000000 38.000000 19.000000 19.000000 21.000000 3.000000 6.000000 1 386.000000 11.000000 4.000000 4.000000 2.000000 10.000000 1.000000 3.000000 4.000000 3.000000 0.000000 2 150.000000 3.000000 6.000000 11.000000 5.000000 7.000000 6.000000 0.000000 4.000000 0.000000 11.000000 3 85.000000 0.000000 6.000000 1.000000 6.000000 1.000000 3.000000 6.000000 4.000000 1.000000 2.000000 4 88.000000 7.000000 5.000000 3.000000 3.000000 8.000000 6.000000 4.000000 1.000000 3.000000 6.000000 5 48.000000 8.000000 5.000000 2.000000 0.000000 0.000000 2.000000 2.000000 6.000000 3.000000 14.000000 6 34.000000 10.000000 6.000000 3.000000 2.000000 2.000000 4.000000 9.000000 8.000000 6.000000 10.000000 7 16.000000 0.000000 2.000000 2.000000 4.000000 1.000000 8.000000 0.000000 6.000000 3.000000 2.000000 8 10.000000 4.000000 4.000000 1.000000 6.000000 3.000000 1.000000 2.000000 1.000000 2.000000 7.000000 9 2.000000 6.000000 3.000000 2.000000 2.000000 5.000000 4.000000 3.000000 1.000000 2.000000 15.000000 10 6.000000 1.000000 8.000000 6.000000 5.000000 7.000000 7.000000 6.000000 8.000000 5.000000 0.000000

#### small population divergence with high migration (50%)

Here the probability of migration between populations is 50%, which essentially erases the effect of population structure. The probability of sampling a parent from the *other lineage* is 50%. This runs a fair bit slower than without migration. 

In [118]:
# define two populations and number of samples from each
populations = [
    ms.PopulationConfiguration(sample_size=10, initial_size=1000),
    ms.PopulationConfiguration(sample_size=10, initial_size=1000),
]

# demographic events (populations divergence time in generations)
demography = [
    ms.MassMigration(time=20000, source=0, dest=1),
]

# migration between populations (set to 0)
migmat = [[0, 0.5], [0.5, 0]]

# get simulator
tree_sequence = ms.simulate(
    migration_matrix=migmat,
    population_configurations=populations,
    demographic_events=demography,
    mutation_rate=1e-7,
    recombination_rate=1e-7,
    length=500000,
)

# get newick string of the tree from each tree_sequence
tree = [i.newick() for i in tree_sequence.trees()]

# draw the trees from all simulations
toytree.tree(tree[0]).draw(layout='down');

<svg class="toyplot-canvas-Canvas" height="260.0px" id="tdec10be3db9e4c6eb9fa0097d8f15db2" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 360.0 260.0" width="360.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 20 9 2 16 8 7 5 13 11 15 17 18 19 4 1 12 6 14 10 3

In [119]:
afs = tree_sequence.allele_frequency_spectrum(
        sample_sets=[range(10), range(10,20)],
        polarised=True,
        span_normalise=False,        
)

In [120]:
import toyplot
toyplot.matrix(afs, width=500, height=500);

<svg class="toyplot-canvas-Canvas" height="500.0px" id="t92e700d3c4af426ba679658b5eb2464e" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 500.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 1 2 3 4 5 6 7 8 9 10 0 0.000000 208.000000 34.000000 17.000000 1.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 1 183.000000 128.000000 53.000000 22.000000 11.000000 11.000000 1.000000 1.000000 0.000000 0.000000 0.000000 2 39.000000 63.000000 36.000000 45.000000 25.000000 19.000000 2.000000 0.000000 0.000000 0.000000 0.000000 3 34.000000 26.000000 24.000000 26.000000 31.000000 6.000000 5.000000 4.000000 2.000000 0.000000 0.000000 4 6.000000 11.000000 4.000000 18.000000 6.000000 12.000000 10.000000 4.000000 1.000000 2.000000 0.000000 5 1.000000 5.000000 5.000000 7.000000 17.000000 20.000000 7.000000 0.000000 1.000000 5.000000 0.000000 6 1.000000 3.000000 4.000000 8.000000 11.000000 9.000000 11.000000 5.000000 5.000000 7.000000 0.000000 7 0.000000 0.000000 0.000000 3.000000 8.000000 1.000000 12.000000 7.000000 8.000000 3.000000 6.000000 8 0.000000 0.000000 0.000000 1.000000 2.000000 4.000000 12.000000 16.000000 7.000000 12.000000 1.000000 9 0.000000 0.000000 0.000000 0.000000 0.000000 3.000000 7.000000 6.000000 8.000000 2.000000 9.000000 10 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 1.000000 4.000000 1.000000 12.000000 0.000000

#### small population divergence with low migration (0.1%)

Here the probability of migration between populations is 1%, which is a more realistic "divergence with gene flow" scenario. 

In [121]:
# define two populations and number of samples from each
populations = [
    ms.PopulationConfiguration(sample_size=10, initial_size=1000),
    ms.PopulationConfiguration(sample_size=10, initial_size=1000),
]

# demographic events (populations divergence time in generations)
demography = [
    ms.MassMigration(time=20000, source=0, dest=1),
]

# migration between populations (set to 0)
migmat = [[0, 0.001], [0.001, 0]]

# get simulator
tree_sequence = ms.simulate(
    migration_matrix=migmat,
    population_configurations=populations,
    demographic_events=demography,
    mutation_rate=1e-7,
    recombination_rate=1e-7,
    length=500000,
)

# get newick string of the tree from each tree_sequence
tree = [i.newick() for i in tree_sequence.trees()]

# draw the trees from all simulations
toytree.tree(tree[0]).draw(layout='down');

<svg class="toyplot-canvas-Canvas" height="260.0px" id="t4a1f71b57e104ef4a64cb368ebf11c48" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 360.0 260.0" width="360.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 18 2 19 16 15 17 3 1 7 12 14 13 11 20 6 4 9 5 10 8

In [122]:
afs = tree_sequence.allele_frequency_spectrum(
        sample_sets=[range(10), range(10,20)],
        polarised=True,
        span_normalise=False,        
)

In [123]:
import toyplot
toyplot.matrix(afs, width=500, height=500);

<svg class="toyplot-canvas-Canvas" height="500.0px" id="tcd27c7c4da424d7aaf172c0c2bb8ef28" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 500.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 1 2 3 4 5 6 7 8 9 10 0 0.000000 165.000000 68.000000 33.000000 32.000000 13.000000 16.000000 1.000000 0.000000 0.000000 0.000000 1 190.000000 50.000000 27.000000 17.000000 8.000000 18.000000 7.000000 4.000000 2.000000 1.000000 0.000000 2 67.000000 41.000000 11.000000 32.000000 10.000000 13.000000 15.000000 13.000000 5.000000 0.000000 1.000000 3 49.000000 34.000000 13.000000 20.000000 6.000000 5.000000 11.000000 5.000000 2.000000 3.000000 0.000000 4 22.000000 12.000000 19.000000 10.000000 5.000000 10.000000 10.000000 16.000000 5.000000 1.000000 1.000000 5 9.000000 17.000000 9.000000 31.000000 5.000000 6.000000 2.000000 27.000000 1.000000 5.000000 5.000000 6 6.000000 4.000000 2.000000 19.000000 5.000000 7.000000 5.000000 10.000000 5.000000 15.000000 0.000000 7 1.000000 12.000000 2.000000 17.000000 3.000000 8.000000 1.000000 8.000000 1.000000 6.000000 0.000000 8 2.000000 2.000000 2.000000 7.000000 4.000000 9.000000 6.000000 19.000000 6.000000 11.000000 5.000000 9 0.000000 4.000000 0.000000 2.000000 4.000000 10.000000 0.000000 8.000000 4.000000 7.000000 13.000000 10 0.000000 0.000000 0.000000 1.000000 2.000000 2.000000 2.000000 7.000000 2.000000 5.000000 0.000000

#### One large, one small pop; deep divergence, low asymmetric migration (1% and 0.1%)

Here the probability of migration between populations is 1%, which is a more realistic "divergence with gene flow" scenario. 

In [124]:
# define two populations and number of samples from each
populations = [
    ms.PopulationConfiguration(sample_size=10, initial_size=1000),
    ms.PopulationConfiguration(sample_size=10, initial_size=1000000),
]

# demographic events (populations divergence time in generations)
demography = [
    ms.MassMigration(time=200000, source=0, dest=1),
]

# migration between populations (set to 0)
migmat = [[0, 0.001], [0.01, 0]]

# get simulator
tree_sequence = ms.simulate(
    migration_matrix=migmat,
    population_configurations=populations,
    demographic_events=demography,
    mutation_rate=1e-7,
    recombination_rate=1e-7,
    length=500000,
)

# get newick string of the tree from each tree_sequence
tree = [i.newick() for i in tree_sequence.trees()]

# draw the trees from all simulations
toytree.tree(tree[0]).draw(layout='down');

<svg class="toyplot-canvas-Canvas" height="260.0px" id="tedf44dbc7a154765b6bf64e2d17f8dd6" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 360.0 260.0" width="360.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 13 10 3 9 7 1 5 2 16 8 19 18 17 15 4 20 14 12 11 6

In [125]:
afs = tree_sequence.allele_frequency_spectrum(
        sample_sets=[range(10), range(10,20)],
        polarised=True,
        span_normalise=False,        
)
toyplot.matrix(afs, width=500, height=500);

<svg class="toyplot-canvas-Canvas" height="500.0px" id="t8035276369e64f9e80218905dabb00a8" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 500.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 1 2 3 4 5 6 7 8 9 10 0 0.000000 209.000000 54.000000 9.000000 9.000000 6.000000 0.000000 0.000000 0.000000 0.000000 0.000000 1 108.000000 50.000000 24.000000 20.000000 1.000000 3.000000 0.000000 0.000000 0.000000 0.000000 0.000000 2 36.000000 16.000000 20.000000 14.000000 14.000000 12.000000 4.000000 2.000000 0.000000 0.000000 0.000000 3 19.000000 17.000000 8.000000 18.000000 16.000000 11.000000 3.000000 0.000000 0.000000 0.000000 0.000000 4 13.000000 6.000000 12.000000 3.000000 4.000000 1.000000 6.000000 0.000000 1.000000 3.000000 4.000000 5 13.000000 14.000000 4.000000 4.000000 5.000000 1.000000 3.000000 2.000000 4.000000 4.000000 5.000000 6 5.000000 1.000000 5.000000 2.000000 9.000000 6.000000 4.000000 2.000000 2.000000 0.000000 1.000000 7 0.000000 4.000000 2.000000 1.000000 4.000000 5.000000 5.000000 7.000000 2.000000 1.000000 3.000000 8 0.000000 0.000000 6.000000 3.000000 0.000000 7.000000 5.000000 7.000000 5.000000 0.000000 2.000000 9 0.000000 0.000000 0.000000 0.000000 0.000000 1.000000 1.000000 1.000000 1.000000 2.000000 12.000000 10 0.000000 0.000000 0.000000 0.000000 0.000000 1.000000 3.000000 1.000000 5.000000 2.000000 0.000000